<a href="https://colab.research.google.com/github/bakamanuke/dounatterunoo/blob/main/claude%E8%A8%88%E7%AE%97%E6%95%B0%E5%AD%A6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math

# 元の行列
A = np.array([[5.0, 2.0, -1.0, 1.0],
              [2.0, -1.0, 3.0, -1.0],
              [-1.0, 3.0, 4.0, 2.0],
              [1.0, -1.0, 2.0, -4.0]])

# 元のQmake関数（3重対角化用）- これは正しく動作する
def Qmake(mat, k):
    row, line = np.shape(mat)
    y = mat[1:, 0]
    if np.linalg.norm(y) == 0:
        return np.eye(row)
    I = np.eye(row-1)
    e = np.array([1])
    for j in range(row-2):
        e = np.append(e, 0)
    s = np.linalg.norm(y) * pow(-1.0, k+1)
    w = s * e
    u = (w - y) / (np.linalg.norm(w - y))
    Q = I - 2 * np.outer(u, u)
    return Q

# 修正版QR分解関数
def QRbunkai_corrected(A):
    """修正版QR分解関数"""
    B = A.copy()
    n = A.shape[0]
    Q = np.eye(n)

    for k in range(n-1):
        # k列目のk行目以降を取得
        x = A[k:, k].copy()

        if np.linalg.norm(x) < 1e-14:
            continue

        # ハウスホルダーベクトルを計算
        alpha = -np.sign(x[0]) * np.linalg.norm(x)
        if abs(alpha) < 1e-14:
            continue

        v = x.copy()
        v[0] = v[0] - alpha

        if np.linalg.norm(v) < 1e-14:
            continue

        v = v / np.linalg.norm(v)

        # ハウスホルダー行列を適用
        H = np.eye(n-k) - 2 * np.outer(v, v)

        # 行列に適用
        A[k:, k:] = H @ A[k:, k:]

        # Q行列を更新
        Q_full = np.eye(n)
        Q_full[k:, k:] = H
        Q = Q @ Q_full.T

    return Q, A

print("=== 元の行列 ===")
print(A)
print("真の固有値:", np.sort(np.linalg.eigvals(A)))
print()

print("=== 3重対角化 ===")
A_work = A.copy()
yoko, tate = np.shape(A_work)
ipsi = 1.0e-12

H = []
for n in range(yoko-2):
    P = np.eye(yoko)
    B = A_work[n:, n:]
    P[n+1:, n+1:] = Qmake(B, n)
    H.append(P)
    A_work = P.T @ A_work @ P

print("3重対角行列:")
print(A_work)
print("3重対角行列の固有値（検証）:", np.sort(np.linalg.eigvals(A_work)))
print()

# 小さな要素を0にする処理
for i in range(yoko):
    for j in range(yoko):
        if abs(A_work[i, j]) < ipsi:
            A_work[i, j] = 0

print("クリーンアップ後:")
print(A_work)
print()

print("=== 修正版QR法の実行 ===")
A_qr = A_work.copy()
max_iterations = 100

for iteration in range(max_iterations):
    Q, R = QRbunkai_corrected(A_qr)
    A_qr = R @ Q

    # 非対角要素の大きさをチェック
    off_diagonal_norm = 0
    for i in range(yoko):
        for j in range(yoko):
            if i != j:
                off_diagonal_norm += abs(A_qr[i, j])

    if iteration % 10 == 0:
        current_eigenvalues = np.diag(A_qr)
        print(f"反復 {iteration}: 非対角要素の和 = {off_diagonal_norm:.2e}")
        print(f"  現在の固有値: {np.sort(current_eigenvalues)}")

    if off_diagonal_norm < 1e-10:
        print(f"収束しました（反復回数: {iteration+1}）")
        break

print("\n=== 最終結果 ===")
print("最終行列:")
print(A_qr)
print()

final_eigenvalues = np.diag(A_qr)
true_eigenvalues = np.linalg.eigvals(A)

print("結果の比較:")
print("計算された固有値:", np.sort(final_eigenvalues))
print("真の固有値:      ", np.sort(true_eigenvalues))
print("誤差:            ", np.abs(np.sort(final_eigenvalues) - np.sort(true_eigenvalues)))
print()

# 誤差の評価
errors = np.abs(np.sort(final_eigenvalues) - np.sort(true_eigenvalues))
print("各固有値の精度評価:")
for i, error in enumerate(errors):
    status = "✓" if error < 1e-6 else "✗"
    print(f"  λ_{i+1}: 誤差 = {error:.2e} {status}")

print("\n=== 元のQR分解関数との比較 ===")
# 元のQR分解関数
def QRbunkai_original(A):
    B = A.copy()
    row, line = np.shape(A)
    C = np.eye(row)
    sankaku = []

    for n in range(row-1):
        I = np.eye(row)
        yoko, tate = np.shape(A)
        e = np.array([1])
        for i in range(yoko-1):
            e = np.append(e, 0)

        if (np.linalg.norm(A[0:, 0]) > A[0, 0]) and (A[0, 0] != 0):
            sigma = (A[0, 0] / abs(A[0, 0])) * np.linalg.norm(A[0:, 0])
            v = (A[0:, 0] + sigma * e) / np.linalg.norm(A[0:, 0] + sigma * e)
        elif (np.linalg.norm(A[0:, 0]) > A[0, 0]) and (A[0, 0] == 0):
            sigma = np.linalg.norm(A[0:, 0])
            v = (A[0:, 0] + sigma * e) / np.linalg.norm(A[0:, 0] + sigma * e)
        else:
            sigma = -A[0, 0]
            v = 0 * e  # ←問題箇所

        H = np.eye(row-n) - 2 * np.outer(v, v)
        A = H @ A
        I[n:, n:] = H
        sankaku.append(I)
        A = A[1:, 1:]  # ←問題箇所

    for i in range(len(sankaku)):
        C = sankaku[i] @ C

    return [C.T, C @ B]

# 元のQR分解でQR法を実行
print("元のQR分解関数でQR法を実行:")
A_qr_orig = A_work.copy()
for iteration in range(50):
    Q_orig, R_orig = QRbunkai_original(A_qr_orig)
    A_qr_orig = R_orig @ Q_orig

    if iteration == 49:  # 最後の反復結果を表示
        orig_eigenvalues = np.diag(A_qr_orig)
        print(f"元の実装の結果: {np.sort(orig_eigenvalues)}")
        print(f"元の実装の誤差: {np.abs(np.sort(orig_eigenvalues) - np.sort(true_eigenvalues))}")
        break

=== 元の行列 ===
[[ 5.  2. -1.  1.]
 [ 2. -1.  3. -1.]
 [-1.  3.  4.  2.]
 [ 1. -1.  2. -4.]]
真の固有値: [-5.65685425 -1.60555128  5.60555128  5.65685425]

=== 3重対角化 ===
3重対角行列:
[[ 5.00000000e+00 -2.44948974e+00  0.00000000e+00  0.00000000e+00]
 [-2.44948974e+00 -4.00000000e+00  1.82574186e+00 -1.24537002e-16]
 [ 0.00000000e+00  1.82574186e+00 -2.60000000e+00 -3.26598632e-01]
 [ 0.00000000e+00 -5.40870636e-16 -3.26598632e-01  5.60000000e+00]]
3重対角行列の固有値（検証）: [-5.65685425 -1.60555128  5.60555128  5.65685425]

クリーンアップ後:
[[ 5.         -2.44948974  0.          0.        ]
 [-2.44948974 -4.          1.82574186  0.        ]
 [ 0.          1.82574186 -2.6        -0.32659863]
 [ 0.          0.         -0.32659863  5.6       ]]

=== 修正版QR法の実行 ===
反復 0: 非対角要素の和 = 7.72e+00
  現在の固有値: [-5.09431828 -1.57811457  5.19354839  5.47888446]
反復 10: 非対角要素の和 = 4.59e+00
  現在の固有値: [-5.20729795 -1.60555127  5.20124945  5.61159977]
反復 20: 非対角要素の和 = 4.62e+00
  現在の固有値: [-5.19772072 -1.60555128  5.19256779  5.61070421]
反復 